In [ ]:
from d123.training.models.sim_agent.smart.smart import SMART
from d123.training.models.sim_agent.smart.smart_config import SMARTConfig

In [ ]:
config = SMARTConfig()
smart_model = SMART(config)

In [ ]:
from pathlib import Path
from d123.dataset.scene.arrow_scene import ArrowScene
from d123.common.visualization.matplotlib.plots import plot_scene_at_iteration

from d123.dataset.scene.scene_builder import ArrowSceneBuilder
from d123.dataset.scene.scene_filter import SceneFilter

from d123.common.multithreading.worker_sequential import Sequential


log_names = None
split = "nuplan_mini_val"
scene_filter = SceneFilter(
    split_names=[split], log_names=log_names, timestamp_threshold_s=8.0, duration_s=8.0, history_s=1.0
)
scene_builder = ArrowSceneBuilder("/home/daniel/d123_workspace/data")
worker = Sequential()
# worker = RayDistributed()
scenes = scene_builder.get_scenes(scene_filter, worker)
scene = scenes[23]
plot_scene_at_iteration(scene, iteration=0)

In [ ]:
from d123.training.feature_builder.smart_feature_builder_ import SMARTFeatureBuilder

feature_builder=  SMARTFeatureBuilder()

feature_dict = feature_builder.build_features(scene)
feature_dict["pt_token"].keys()

In [ ]:
from matplotlib import pyplot as plt


fig, ax = plt.subplots(figsize=(10, 10))

traj_pos = feature_dict["map_save"]["traj_pos"]
pl_type = feature_dict["pt_token"]["pl_type"]
print(traj_pos.shape)
for i in range(
    traj_pos.shape[0],
):
    if pl_type[i] == 1:
        ax.plot(traj_pos[i, :, 0], traj_pos[i, :, 1])

ax.set_title("map_save/traj_pos")

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 10))

# for boundary in boundaries:
#     ax.plot(boundary[:, 0], boundary[:, 1], color="blue", linewidth=1.0, alpha=1.0)
for map_segment in map_segments:
    ax.plot(map_segment[:, 0], map_segment[:, 1], linewidth=1.0, alpha=1.0)
